In [8]:
import csv
import logging
import os
import urllib3
import json

from google.cloud import storage

In [9]:
CRYPTO_URL = 'https://api.coinmarketcap.com/v1/ticker/'
GCS_BUCKET = 'anant-caserta-eval'

In [10]:
def download_crypto_data(url):
  http = urllib3.PoolManager()
  response = http.request('GET', url)
  return json.loads(response.data.decode('utf-8'))

In [11]:
crypto_data = download_crypto_data(CRYPTO_URL)

/usr/local/envs/py3env/lib/python3.5/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [12]:
crypto_data

[{'24h_volume_usd': '9677140000.0',
  'available_supply': '16997250.0',
  'id': 'bitcoin',
  'last_updated': '1524605675',
  'market_cap_usd': '160819990792',
  'max_supply': '21000000.0',
  'name': 'Bitcoin',
  'percent_change_1h': '0.41',
  'percent_change_24h': '6.2',
  'percent_change_7d': '18.65',
  'price_btc': '1.0',
  'price_usd': '9461.53',
  'rank': '1',
  'symbol': 'BTC',
  'total_supply': '16997250.0'},
 {'24h_volume_usd': '3427170000.0',
  'available_supply': '99025563.0',
  'id': 'ethereum',
  'last_updated': '1524605661',
  'market_cap_usd': '69876497124.0',
  'max_supply': None,
  'name': 'Ethereum',
  'percent_change_1h': '0.13',
  'percent_change_24h': '10.25',
  'percent_change_7d': '39.03',
  'price_btc': '0.0745234',
  'price_usd': '705.641',
  'rank': '2',
  'symbol': 'ETH',
  'total_supply': '99025563.0'},
 {'24h_volume_usd': '1448950000.0',
  'available_supply': '39146203398.0',
  'id': 'ripple',
  'last_updated': '1524605643',
  'market_cap_usd': '37385368023.0

In [16]:
def crypto_data_to_csv(crypto_data, file_path):
  with open(file_path, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, crypto_data[0].keys())
    for row in crypto_data:
      writer.writerow(row)

In [17]:
def crypto_to_gcs(data_csvpath, bucket, blobname):
  client = storage.Client()
  bucket = client.get_bucket(bucket)
  blob = bucket.blob(blobname)
  blob.upload_from_filename(data_csvpath)

In [19]:
file_path = '/tmp/crypto_data.csv'
crypto_data_to_csv(crypto_data, file_path)
crypto_to_gcs(file_path, GCS_BUCKET, 'crypto_data.csv')

In [22]:
schema = ['{}:{}'.format(field, 'STRING') for field in crypto_data[0].keys()]
','.join(schema)

'available_supply:STRING,name:STRING,percent_change_1h:STRING,percent_change_24h:STRING,last_updated:STRING,market_cap_usd:STRING,symbol:STRING,24h_volume_usd:STRING,price_usd:STRING,percent_change_7d:STRING,total_supply:STRING,id:STRING,price_btc:STRING,rank:STRING,max_supply:STRING'